# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



# Ex1 : Part 1 to 7, Model and solver

Our goal is to solve the following optimisation problem. for that we will make a text file for the coeffients and use it to solve the problem. 



 \begin{array}{l}
max\ \ \ -0.5x_{1} \ −\ 1.25x_{2} \ −\ 5x_{3} \ \ +\ 3x_{4} \ +\ 10x_{5} \ \ +\ 25x_{6} \ \\
\\
s.t\ \ \ \ \ 0.8x_{1} \ -1.3x_{2} \ -x_{4} \ \leqslant \ 12\\
x_{2} \ -\ x_{3} -x_{5} \ \leqslant \ 0.6\\
x_{3} -x_{6\ } \leqslant \ 1.2\\
0.5x_{1} +0.8x_{2} \ +4x_{3} \ \leqslant \ 45\\
0.9x_{1} +1.5x_{2} \ \ \leqslant \ 27\\
\\
xi\ \geq \ 0\ \forall i\ \in \ \{1,\ 2,\ .\ .\ .\ ,\ 5\} .\ 
\end{array}


In [ ]:
# Creating model
model = ConcreteModel()

#importing the txt file of coefficients
coeff = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')


# Numbers of variable N, and number of constraints M
N = coeff.shape[1] - 1
M = coeff.shape[0] -1


col_index = np.arange(N)
row_index = np.arange(M)

#Coefficients of constraints matrix with rhs
coeff_cons_with_rhs = coeff[1:,:]



# creating variables and their type as binary
model.x = Var(col_index)


# Initialising list of constraints
model.cons = ConstraintList()

# Model objective
model.obj = Objective(expr = summation(coeff[0,:-1],model.x), sense = maximize)

#Adding constraits and bounds

for i in row_index:
  model.cons.add(summation(coeff_cons_with_rhs[i,:-1],model.x) <= coeff_cons_with_rhs[i,-1])


for i in col_index: 
  model.x[i].setlb(0)

#Printing the model

model.pprint()

# Solving the model with cbc solver
cbc_solver = SolverFactory('cbc')
result = cbc_solver.solve(model)

print(result)
# Printing result
print('\n')
print('\n')

print('Status : ', result.solver.Status)
print('Terminating condition : ', result.solver.termination_condition)
print('Message : ', result.solver.termination_message)
print('\n')



2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : - 0.5*x[0] - 1.25*x[1] - 5.0*x[2] + 3.0*x[3] 

# Ex1 : Part 8 , 9

***Devising a remedy and solving again.***

Now since in the previous part we saw that the problem is unbounded. The reason the solution is unbounded because the region is not bounded above for the maximising problem i.e we will keep on getting the optimal value and there will always be more optimal value than that. 


Thats why we try this problem with minimising the problem. In that case we will find the solution and the values are printed below.


In [ ]:
model.obj.set_sense(minimize)


# Solving the model with cbc solver
cbc_solver = SolverFactory('cbc')
result = cbc_solver.solve(model)




print('Status : ', result.solver.Status)
print('Terminating condition : ', result.solver.termination_condition)
print('Message : ', result.solver.termination_message)
print('The optimal value of the objective function is : ', model.obj())
print('\n')

print('The value of the variables are : ')
for i in col_index:
  print('x[',i,'] : ', model.x[i].value)
print('\n')

print('And the value of the constraints are')

model.cons.display()



Status :  ok
Terminating condition :  optimal
Message :  Model was solved to optimality (subject to tolerances), and an optimal solution is available.
The optimal value of the objective function is :  -17.2125


The value of the variables are : 
x[ 0 ] :  17.925
x[ 1 ] :  1.8
x[ 2 ] :  1.2
x[ 3 ] :  0.0
x[ 4 ] :  0.0
x[ 5 ] :  0.0


And the value of the constraints are
cons : Size=5
    Key : Lower : Body               : Upper
      1 :  None : 12.000000000000002 :  12.0
      2 :  None : 0.6000000000000001 :   0.6
      3 :  None :                1.2 :   1.2
      4 :  None :            15.2025 :  45.0
      5 :  None :            18.8325 :  27.0
